## spark data 처리

1. 로컬 stock_min 정보를 읽어 warehouse구성
2. 구성 위치는 1차로 로컬 dw 하위로 stocklab db 를 구성
3. 이후 google drive로 위치 변경 

In [6]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.types import *
from pyspark.sql.functions import lit
 

    
db_path = 'd:/dw/data-warehouse/stocklab_db'

spark = SparkSession.builder.appName('test_spark') \
  .config("spark.sql.warehouse.dir", db_path) \
  .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
  .enableHiveSupport() \
  .getOrCreate()

from delta.tables import *
from pyspark.sql.functions import *

In [9]:
#spark.sql("drop database if exists stocklab CASCADE")
#AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: InvalidOperationException(message:Database stocklab is not empty. One or more tables exist.)
#force drop cascade
spark.sql("create database IF NOT EXISTS stocklab")
spark.sql("show databases ").show()
spark.sql("describe database stocklab").show(100,False)

+---------+
|namespace|
+---------+
|  default|
| stocklab|
+---------+

+-------------------------+--------------------------------------------------+
|database_description_item|database_description_value                        |
+-------------------------+--------------------------------------------------+
|Database Name            |stocklab                                          |
|Comment                  |                                                  |
|Location                 |file:/d:/dw/data-warehouse/stocklab_db/stocklab.db|
|Owner                    |Administrator                                     |
+-------------------------+--------------------------------------------------+



In [10]:
df_min = spark.read.format("parquet").load("d:/dw/stock_min")

In [12]:
df_min.show()

+-----+----+---------+----+----+------+-----+--------+------+
|close|high|jdiff_vol| low|open|  time|value|    date|shcode|
+-----+----+---------+----+----+------+-----+--------+------+
| 3915|3965|   373432|3900|3960|090100| 1475|20210408|303030|
| 3895|3945|   197449|3890|3915|090200|  775|20210408|303030|
| 3875|3900|   101317|3860|3895|090300|  394|20210408|303030|
| 3895|3900|    63951|3875|3875|090400|  249|20210408|303030|
| 3910|3915|    86320|3885|3895|090500|  336|20210408|303030|
| 3940|3940|   146708|3905|3910|090600|  576|20210408|303030|
| 3930|3945|   140699|3920|3940|090700|  553|20210408|303030|
| 3890|3935|   100168|3890|3920|090800|  391|20210408|303030|
| 3920|3935|    75051|3890|3895|090900|  294|20210408|303030|
| 3965|3980|   330749|3915|3925|091000| 1310|20210408|303030|
| 4005|4015|   466790|3970|3970|091100| 1865|20210408|303030|
| 4020|4020|   291048|3990|4000|091200| 1165|20210408|303030|
| 4040|4050|   438413|4010|4015|091300| 1770|20210408|303030|
| 4040|4

In [ ]:
df_min.groupBy("shcode").agg(count("shcode")).show()